<a href="https://colab.research.google.com/github/jumafernandez/clasificacion_correos/blob/main/notebooks/jaiio/00-procesamiento_correos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de correos electrónicos

El backup de correos se encuentra en un archivo pst. Los archivos en el formato PST son archivos de datos creados por Microsoft Outlook, un gestor de información personal y una parte de la suite Microsoft Office.

## 1. Primeros pasos

### 1.1 Instalación de librerías

In [1]:
!pip install libpff-python
!pip install -U -q PyDrive

### 1.2 Funciones propias

Se definen funciones propias para la limpieza de los correos y las cadenas de texto:

In [16]:
#%% Función para limpiar las consultas
def limpiar_correo(text):
    '''Se limpian las cadenas de texto'''
    
    # Paso a minusculas
    text = str(text).lower()

    # Reemplazo los tildes
    text = text.replace("\\xc3\\xa1", "á")
    text = text.replace("\\xc3\\xa9", "é")
    text = text.replace("\\xc3\\xad", "í")
    text = text.replace("\\xc3\\xb3", "ó")
    text = text.replace("&uacute;", "ú")
    text = text.replace("\\xc3\\xb1", "ñ")
    text = text.replace("&aacute;", "á")
    text = text.replace("&eacute;", "é")
    text = text.replace("&iacute;", "í")
    text = text.replace("&oacute;", "ó")
    text = text.replace("&uacute;", "ú")
    text = text.replace("&ntilde;", "ñ")
    text = text.replace("&ordm", "°")

    # Quito los fin de linea y caracteres especiales
    text = text.replace("\\n", " ")
    text = text.replace("\\r", "")
    text = text.replace("\\", "")
    text = text.replace("b\'", "")

    return text

def limpiar_consulta(text):
    '''Se limpian las consultas'''   
    # Separo la consulta en encabezado y cuerpo
    COMIENZO_CORREO = "de: u.n.lu. [mailto:consultasweb@mail.unlu.edu.ar]"
    GUIONES_CUERPO  = "-------------------------"
    
    text = str(text).replace(" >", "")
    text = text.split("---------------------------------")
    if len(text)>1:
        # El encabezado solo posee la fecha como dato importante
        encabezado = text[0]
        encabezado = encabezado.replace(COMIENZO_CORREO, "")
        #fecha = encabezado[len(encabezado)-len("08.23.2015-00:57:19")-2:len(encabezado)].strip()
        inicio_fecha = encabezado.find("enviado :")
        fecha        = encabezado[inicio_fecha+len("enviado :"):len(encabezado)].strip()
        fecha        = fecha[0:len("08.20.2019-20:48:53")]
        hora         = fecha.split("-")[1]
        fecha        = fecha.split("-")[0].replace(".", "-")
        
        # Cuerpo        
        cuerpo = text[1]
        cuerpo = cuerpo.replace(GUIONES_CUERPO, "")
        
        # Busco el inicio de cada dato para estructurarlos
        inicio_ap_nom    = cuerpo.find("nombre y apellido: ")
        inicio_legajo    = cuerpo.find("legajo: ")
        inicio_documento = cuerpo.find("documento: ")
        inicio_carrera   = cuerpo.find("carrera: ")
        inicio_telefono  = cuerpo.find("teléfono: ")
        inicio_email     = cuerpo.find("e-mail: ")
        inicio_consulta  = cuerpo.find("mensaje / consulta: ")

        apellido_nombre = cuerpo[inicio_ap_nom+len("nombre y apellido: "):inicio_legajo-1]
        legajo          = cuerpo[inicio_legajo+len("legajo: "):inicio_documento-1]
        documento       = cuerpo[inicio_documento+len("documento: "):inicio_carrera-1]
        carrera         = cuerpo[inicio_carrera+len("carrera: "):inicio_telefono-1]
        telefono        = cuerpo[inicio_telefono+len("teléfono: "):inicio_email-1]
        email           = cuerpo[inicio_email+len("e-mail: "):inicio_consulta-1]
        consulta        = cuerpo[inicio_consulta+len("mensaje / consulta: "):len(cuerpo)]
        
    else:
        fecha = text[0]
        hora  = -1
        apellido_nombre = -1
        legajo          = -1
        documento       = -1
        carrera         = -1
        telefono        = -1
        email           = -1
        consulta        = -1
        
    return fecha, hora, apellido_nombre, legajo, documento, carrera, telefono, email, consulta

### 1.3 Descarga de archivo de correos

Se abre el archivo con las consultas que se encuentra en Google Drive (~=800 mb)

In [17]:
import pypff
from google.colab import drive
drive.mount('/content/drive')

pst_file = 'drive/MyDrive/Consultas.pst'

pst = pypff.file()
pst.open(pst_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Se verifican las carpetas que se encuentran dentro del backup y la cantidad de mensajes de cada una de ellas:

In [18]:
root_node = pst.get_root_folder()
carpeta_Outlook = root_node.get_sub_folder(1)
print('Carpetas del Outlook:')
for i in range(0, carpeta_Outlook.get_number_of_sub_folders()):
    folder=carpeta_Outlook.get_sub_folder(i)
    print(str(i) + "-" + folder.get_name() + ": " + str(folder.get_number_of_sub_messages()))

Carpetas del Outlook:
0-Elementos eliminados: 1491
1-Bandeja de entrada: 22703
2-Bandeja de salida: 0
3-Elementos enviados: 24166
4-Correo electrónico no deseado: 19
5-Configuración de acción de conversación: 0
6-Configuración de pasos rápidos: 0
7-Calendario: 0
8-Diario: 0
9-Tareas: 0
10-Infected Items: 0
11-Contactos: 0
12-Notas: 0
13-Borrador: 0
14-Fuentes RSS: 0


## 2. Procesamiento de correos

Se escoge trabajar con los correos enviados puesto que los mismos cuentan con la consulta y su respuesta (eventual etiqueta):

In [25]:
enviados = carpeta_Outlook.get_sub_folder(3)

Se definen constantes que tienen que ver con el tamaño de la muestra y el nombre del archivo donde se van a guardar los correos procesados:

In [26]:
CANTIDAD_CORREOS = enviados.get_number_of_sub_messages()
print('La muestra posee {} correos.'.format(CANTIDAD_CORREOS))
# Constante para la etapa de pruebas
# CANTIDAD_CORREOS = 10

# Datos del archivo donde se guardan las consultas
PATH_ARCHIVO_CORREOS = "correos-procesados.csv"

La muestra posee 24166 correos.


Se definen un conjunto ce constantes que tienen que ver con el texto de los correos:

In [27]:
SEPARADOR_CONSULTA_RESPUESTA = "-----mensaje original-----"
INICIO_DISTINTO = "de: u.n.lu. [mailto:consultasweb@mail.unlu.edu.ar]"

In [28]:
# Se crea el data frame donde se guardan los correos procesados
import pandas as pd
nombre_campos = ["fecha", "hora", "apellido_nombre", "legajo", "documento", "carrera", "telefono", "email", "consulta", "respuesta"]

df_correos = pd.DataFrame(columns = nombre_campos)

In [29]:
for i in range(0, CANTIDAD_CORREOS):
  # Se toma el correo de índice i
  correo = enviados.get_sub_message(i)
  cuerpo = correo.get_plain_text_body()
  if cuerpo:
    
    # Se hace una limpieza inicial del texto
    cuerpo = limpiar_correo(cuerpo)      
    
    # Se separa la consulta de la respuesta, solo acepta un ida y vuelta
    cuerpo = cuerpo.split(SEPARADOR_CONSULTA_RESPUESTA)
    
    # Se deciden desechar los correos no separables, con varios idas/vueltas
    if len(cuerpo)>1:
      respuesta = cuerpo[0]
      consulta = cuerpo[1]
            
      # Si es un ida/vuelta UNICO
      if consulta.find(INICIO_DISTINTO)!=-1:
        fecha, hora, apellido_nombre, legajo, documento, carrera, telefono, email, consulta = limpiar_consulta(consulta)

        if apellido_nombre!=-1:
          
          correo_procesado = {'fecha': fecha, 'hora': hora, 'apellido_nombre': apellido_nombre, 'legajo': legajo, 'documento': documento, 'carrera': carrera, 
                              'telefono': telefono, 'email': email, 'consulta': consulta, 'respuesta': respuesta}
        
          df_correos = df_correos.append(correo_procesado, ignore_index=True)


Se vuelvan los correos a un CSV:

In [31]:
df_correos.to_csv('correos-procesados.csv', index=False)

Si estamos en COLAB lo pasamos a Google Drive:

In [33]:
ENTORNO='Colab'

if ENTORNO='Colab':
  from google.colab import drive
  drive.mount('drive')
  !cp correos-procesados.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
